---
title: "Applying PyTorch"
description: "Using basic PyTorch to process and analyze BRCA data"
author: "Vivek Sriram"
date: "8/19/2024"
date-modified: "8/19/2024"
image: zolaCake.jpg
categories:
  - Tutorials
draft: true
jupyter: python3
---


In today's blog post, I will take the PyTorch foundation I built in my previous post and apply it to a new dataset with a more complicated neural network and additional hyperparameter training.

With background out of the way, let's get started\~

```{{r}}
library(reticulate)
use_python('/opt/anaconda3/bin/python')
```

Import required packages.


In [ ]:
# Import required packages
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

We check to see if we can train our model on a hardware accelerator like the GPU or MPS if available. Otherwise, we'll use the CPU.


In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Data

### Importing and transforming the data

BRCA TCGA Pan-Cancer Atlas dataset: - patients with survival information and gene expression data from cBioPortal - network of biological interactions between those nodes from Pathway commons. - Dataset contains the gene features of 1082 patients, as well as the overall survival time (in months) of each patient as a label.

1082 graphs 9288 nodes 271,771 edges 1082 features

Integrated breast cancer BRCA TCGA data from cBioPortal and a biological network for node connections from Pathway commons.

Data was preprocessed to form a single dataset that could be converted to PyTorch Geometric data ojbects.

Dataset contains gene features of each patient and the overall survival time of each patient


In [ ]:
ppi = torch_geometric.datasets.PPI(
  root='./posts/pytorch-brca/'
)

In [ ]:
brca = torch_geometric.datasets.BrcaTcga(
  root='./posts/pytorch-brca/'
)

In [ ]:
len(brca)
brca.num_classes
brca.num_node_features

Our dataset has 1082 graphs, with 824 classes. Each node in our dataset has 1 feature.


In [ ]:
brca_graph0 = brca[0]

print(brca_graph0)
brca_graph0.is_undirected()

# Access the node features, edge indices, and target label
node_features = brca_graph0.x
edge_index = brca_graph0.edge_index
target = brca_graph0.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

The first graph in our dataset has 9288 nodes, each with a single feature. There are 271,771 directed edges in our graph. And, this data object is holding exactly one graph-level target.


In [ ]:
# Apply 70/30 train/test split
splitIndex = round(len(brca)*.7)

training_data = brca[:splitIndex]
test_data = brca[splitIndex:]

### Preparing data for training with `DataLoaders`

Here we define a batch size of 64 - each element in the `DataLoader` iterable will return a batch of 64 features and labels.


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(
  training_data, 
  batch_size=batch_size, 
  shuffle=True
)

for step, data in enumerate(train_dataloader):
    data = data.to(device)

test_dataloader = DataLoader(
  test_data, 
  batch_size=batch_size, 
  shuffle=False
)

for step, data in enumerate(test_dataloader):
    data = data.to(device)

## Modeling

### Defining a neural network


In [ ]:
class GCN(nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final regression layer
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)

        return x.squeeze()  # Remove the extra dimension

In [ ]:
model = GCN(hidden_channels=64, num_node_features=1).to(device)
print(f"Model structure: {model}")

In [ ]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

### Optimizing the Model Parameters

#### Hyperparameters


In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#### Optimization Loop


In [ ]:
# Initialize the loss function and the optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Full Implementation


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    for step, data in enumerate(train_dataloader):
        data = data.to(device)

        # Compute prediction and loss
        pred = model(
          data.x, 
          data.edge_index, 
          data.batch
        )
        
        loss = loss_fn(
          pred, 
          data.y.view(-1)
        )

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for step, data in enumerate(test_dataloader):
            data = data.to(device)

            pred = model(
              data.x, 
              data.edge_index, 
              data.batch
            )
            
            loss += loss_fn(
              pred, 
              data.y.view(-1)
            ).item()

    test_loss /= num_batches

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and then backpropagates the prediction error to adjust the model’s parameters.

We can also check the model’s performance against the test dataset to ensure it is learning.

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

We can now use this model to make individual predictions.

To use the model, we pass it the input data. This executes the model’s `forward`, along with some background operations. Note that we do not call `model.forward()` directly!


In [ ]:
model.eval()  # Set the model to evaluation mode

predictions = []  # List to store the predicted outputs
test_losses = []
with torch.no_grad():
    test_loss = 0
    for data in test_loader:
        data = data.to(device)  # Move the batch of data to CUDA device
        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1).double())
        test_loss += loss.item()

        average_test_loss = test_loss / len(test_loader)
        test_losses.append(average_val_loss)

        predictions.append(out.cpu().detach().numpy())  # Convert the predictions to NumPy array

# Concatenate the predictions from multiple batches
predictions = np.concatenate(predictions)

# Print the predictions
print(predictions[:10])
[43.55725092 44.08028075 38.91240359 49.6856782  57.11545471 49.2488646
 46.26797878 69.44459431 40.08783161 47.09477596]
Making Predictions and calculating MSE
# Convert the test data batches to a list of Data objects
test_data_batches = [
    Data(x=batch.x, edge_index=batch.edge_index, y=batch.y) for batch in test_loader
]

# Convert the predictions to PyTorch tensors
predictions = torch.tensor(predictions)

# Convert the ground truth labels of the test data to a PyTorch tensor
y_true = torch.cat([batch.y for batch in test_data_batches])

# Calculate the mean squared error (MSE) loss using PyTorch's function
mse_loss = torch.nn.functional.mse_loss(predictions.view(-1), y_true.view(-1))

print(f"Mean Squared Error (MSE) Loss: {mse_loss:.4f}")
Mean Squared Error (MSE) Loss: 957.8370
Plotting the convergence of the model
# Visualize convergence results
plt.figure()
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Convergence of Model')
plt.show()

# Summary

This concludes my walkthrough of using basic PyTorch to process data and train a neural network. As mentioned, in a future post in the coming weeks, I will iterate on this pipeline, demonstrating a different neural network architecture and more data exploration for a different dataset. Until next time, \[VS\]Coders!